<center>
<h1>Analyzing Bee Taxonomy:</h1>
<h2>Integrating GBIF, NCBI and iNaturalist Data for Apidae Insights</h2>
</center>

![title](https://live.staticflickr.com/4059/4632384645_a2230b26d5_b.jpg)

This Python notebook is designed for the purpose of integrating taxonomic data from three major biological databases, GBIF (Global Biodiversity Information Facility), NCBI (National Center for Biotechnology Information), and iNaturalist to enhance the accuracy and comprehensiveness of ecological and biological research. GBIF primarily focuses on biodiversity data including species distribution and ecological information, whereas NCBI provides a broader range of data including genomic and taxonomic details. iNaturalist, on other hand, is one of the most important Citizen Science project to collect biological data

Combining these sources enables researchers to cross-validate species identifications and improve the richness of ecological datasets with genetic information. A key biological task performed in this notebook is the construction of a taxonomic tree, which helps in visualizing and understanding the evolutionary relationships and classification hierarchy among different species within a chosen taxon (in this case, Apidae - a family of bees).

## 1. Importing libraries

In [ ]:
import pandas as pd
import warnings

pd.set_option('display.max_colwidth', None)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import taxonmatch as txm

## 2. Downloading and processing samples

The initial steps involve downloading the most recent taxonomic data from GBIF and NCBI to ensure the analysis is based on the latest available information. 

In [ ]:
gbif_dataset = txm.download_gbif_taxonomy()

In [ ]:
ncbi_dataset = txm.download_ncbi_taxonomy()

## 3.a Training the classifier model

If required, the notebook outlines steps to train a machine learning classifier to distinguish between correct and incorrect taxonomic matches. This involves generating positive and negative examples, preparing the training dataset, and comparing different models.

In [ ]:
#positive_matches = txm.generate_positive_set(gbif_dataset, ncbi_dataset, 500)

In [ ]:
#negative_matches = txm.generate_negative_set(gbif_dataset, ncbi_dataset, 500)

In [ ]:
#full_training_set = txm.prepare_data(positive_matches, negative_matches)

In [ ]:
#full_training_set.to_csv("training_set.txt", index = False)

In [ ]:
#X_train, X_test, y_train, y_test = txm.generate_training_test(full_training_set)

In [ ]:
#model = txm.XGBClassifier(learning_rate=0.1,n_estimators=500, max_depth=9, n_jobs=-1, colsample_bytree = 1, subsample = 0.8)

In [ ]:
#model.fit(X_train, y_train, verbose=False)

In [ ]:
#model.save_model("xgb_model.json")

## 3.b Load a pre-trained model

Alternatively, it provides the option to load a pre-trained model, simplifying the process for routine analyses.

In [ ]:
model = txm.load_xgb_model()

## 4. Match NCBI with GBIF dataset 

In this section, the focus is on comparing and aligning the taxonomic data from NCBI and GBIF datasets. It specifically targets the taxon "Apidae" to narrow down the analysis to a specific family of bees. Using a pre-trained machine learning model, the notebook matches records from both datasets, categorizing them as exact matches, unmatched, or potentially mislabeled due to typographical errors

In [ ]:
gbif_apidae, ncbi_apidae = txm.select_taxonomic_clade("Apidae", gbif_dataset, ncbi_dataset)

In [ ]:
matched_df, unmatched_df, possible_typos_df = txm.match_dataset(gbif_apidae, ncbi_apidae, model, tree_generation = True)

## 5. Generate the taxonomical tree 

In this section, the notebook constructs a taxonomic tree from the matched and unmatched data between the GBIF and NCBI datasets, focusing on the Apidae family. This visual representation helps to illustrate the evolutionary relationships and classification hierarchy among the species. The tree is then converted into a dataframe for further analysis and saved in textual format for documentation and review purposes.

In [ ]:
apidae_tree = txm.generate_taxonomic_tree(matched_df, unmatched_df)

In [ ]:
df_apidae = txm.convert_tree_to_dataframe(apidae_tree, gbif_apidae, ncbi_apidae, "apidae_taxonomic_tree_df.txt")

In [ ]:
txm.save_tree(apidae_tree, "apidae_tree.txt")

In [ ]:
txm.print_tree(apidae_tree)

## 5.1 Reroot the taxoonomical tree

Alternatively, the previously constructed taxonomic tree will be pruned (rerooted) to focus on a specific lineage within Apidae. By selecting a target taxon—in this case, the genus Bombus—all branches below this level will be retained while discarding unrelated taxa. This approach allows for a more detailed analysis of a specific clade, facilitating a clearer visualization of evolutionary relationships and taxonomic consistency within the selected group.

In [ ]:
bombus_tree = txm.reroot_tree(apidae_tree, root_name="bombus")

In [ ]:
txm.print_tree(bombus_tree)

## 6. Add iNaturalist Information

In this final section, the previously curated dataset, which integrates taxonomic information from both GBIF and NCBI, will be further enriched by incorporating data from iNaturalist. This additional dataset will provide valuable community-driven observations, complementing the existing taxonomy with real-world records contributed by citizen scientists and researchers. The ultimate result of this process will be the construction of a comprehensive taxonomic tree that includes unique identifiers from all three datasets—GBIF, NCBI, and iNaturalist—ensuring a more robust and harmonized representation of Apidae taxonomy.

In [ ]:
inat_dataset = txm.download_inat_taxonomy()

In [ ]:
inat_apidae = txm.select_inat_clade(inat_dataset, "apidae")

In [ ]:
inat_tree = txm.add_inat_taxonomy(apidae_tree, inat_apidae)

In [ ]:
txm.print_tree(inat_tree)

In [ ]:
df_apidae_with_inaturalist = txm.convert_tree_to_dataframe(apidae_tree, gbif_dataset[1], ncbi_dataset[1], "apidae_final_df.txt", inat_dataset=inat_apidae)

In [ ]:
txm.save_tree(inat_tree, "apidae_final_tree.txt", output_format='txt')